<a href="https://colab.research.google.com/github/fongbubble/fongbubble_UoB_EFIMM0142_Individual_Assessment/blob/main/Question1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [ ]:
!pip install pulp
from pulp import *

# Question 1

## Question 1.a

In [ ]:
# Define the problem
model = LpProblem("Question1_a", LpMinimize)

# Parameters
products = ["A", "B", "C"]
months = ["1", "2", "3", "4"]
production_cost = {"A": 20, "B": 30, "C": 10}
holding_cost = {"A": 1, "B": 2, "C": 0.5}
production_hours = {"A": 1.2, "B": 1.3, "C": 1}
available_hours = {"1": 500, "2": 450, "3": 400, "4": 350}
demand = {
    "1": {"A": 100, "B": 80, "C": 120},
    "2": {"A": 110, "B": 90, "C": 130},
    "3": {"A": 120, "B": 100, "C": 140},
    "4": {"A": 130, "B": 110, "C": 150},
}

# Decision Variables
P = {(p, m): LpVariable(f"P_{p}_{m}", lowBound=0, cat="Integer") for p in products for m in months}
I = {(p, m): LpVariable(f"I_{p}_{m}", lowBound=0, cat="Continuous") for p in products for m in months}

# Objective Function: Minimize total production and holding costs
model += lpSum(production_cost[p] * P[p, m] + holding_cost[p] * I[p, m] for p in products for m in months)

# Constraints

# Production and Inventory Balance (strict equality)
for p in products:
    for i, m in enumerate(months):
        prev_inventory = I[p, months[i - 1]] if i > 0 else 0
        model += P[p, m] + prev_inventory == demand[m][p] + I[p, m]

# Production Hours Limit
for m in months:
    model += lpSum(production_hours[p] * P[p, m] for p in products) <= available_hours[m]

# Explicitly ensure P and I are non-negative for all months except m = 3
for p in products:
    for m in months:
        if m != "4":  # Exclude month 3
            model += P[p, m] >= 0
            model += I[p, m] >= 0

# Solve the problem
model.solve()

# Print results
print("Status:", LpStatus[model.status])
print("Optimal Solution:")
for p in products:
    for m in months:
        print(f"{p} - {m}: Produce = {P[p, m].varValue}, Inventory = {I[p, m].varValue}")

# Calculate total cost for each product
product_costs = {p: 0 for p in products}

# Add production and holding costs for each product
for p in products:
    production_cost_sum = sum(
        production_cost[p] * P[p, m].varValue for m in months
    )
    holding_cost_sum = sum(
        holding_cost[p] * I[p, m].varValue for m in months
    )

    # Store total cost for the product
    product_costs[p] = production_cost_sum + holding_cost_sum

# Print total costs for each product
print("\nTotal Cost Breakdown by Product:")
for p in products:
    print(f"Product {p}: £{product_costs[p]:.2f}")

# Optional: Print the overall total cost (should match model.objective.value())
overall_cost = sum(product_costs[p] for p in products)
print(f"\nOverall Total Cost: £{overall_cost:.2f}")


Status: Optimal
Optimal Solution:
A - 1: Produce = 100.0, Inventory = 0.0
A - 2: Produce = 110.0, Inventory = 0.0
A - 3: Produce = 120.0, Inventory = 0.0
A - 4: Produce = 130.0, Inventory = 0.0
B - 1: Produce = 80.0, Inventory = 0.0
B - 2: Produce = 90.0, Inventory = 0.0
B - 3: Produce = 100.0, Inventory = 0.0
B - 4: Produce = 110.0, Inventory = 0.0
C - 1: Produce = 162.0, Inventory = 42.0
C - 2: Produce = 201.0, Inventory = 113.0
C - 3: Produce = 126.0, Inventory = 99.0
C - 4: Produce = 51.0, Inventory = 0.0

Total Cost Breakdown by Product:
Product A: £9200.00
Product B: £11400.00
Product C: £5527.00

Overall Total Cost: £26127.00


## Question 1.b

In [ ]:
# Define the problem
modelb = LpProblem("Question1_b", LpMinimize)

# Parameters
products = ["A", "B", "C"]
months = ["1", "2", "3", "4"]
production_cost = {"A": 20, "B": 30, "C": 10}
holding_cost = {"A": 1, "B": 2, "C": 0.5}
production_hours = {"A": 1.2, "B": 1.3, "C": 1}
available_hours = {"1": 500, "2": 450, "3": 400, "4": 350}
demand = {
    "1": {"A": 100, "B": 80, "C": 120},
    "2": {"A": 110, "B": 90, "C": 130},
    "3": {"A": 120, "B": 100, "C": 140},
    "4": {"A": 130, "B": 110, "C": 150},
}

# Decision Variables
P = {(p, m): LpVariable(f"P_{p}_{m}", lowBound=0, cat="Integer") for p in products for m in months}
I = {(p, m): LpVariable(f"I_{p}_{m}", lowBound=0, cat="Continuous") for p in products for m in months}

# Objective Function: Adjust cost for P_B_3 to 35 instead of 30
modelb += lpSum(
    production_cost[p] * P[p, m] + holding_cost[p] * I[p, m]
    for p in products for m in months
    if not (p == "B" and m == "3")
) + 35 * P["B", "3"]  # Add the adjusted cost for P_B_3

# Constraints
# Production and Inventory Balance (strict equality)
for p in products:
    for i, m in enumerate(months):
        prev_inventory = I[p, months[i - 1]] if i > 0 else 0
        modelb += P[p, m] + prev_inventory == demand[m][p] + I[p, m]

# Production Hours Limit
for m in months:
    modelb += lpSum(production_hours[p] * P[p, m] for p in products) <= available_hours[m]

# Explicitly ensure P and I are non-negative for all months except m = 3
for p in products:
    for m in months:
        if m != "4":  # Exclude month 4
            modelb += P[p, m] >= 0
            modelb += I[p, m] >= 0

# Solve the problem
modelb.solve()

# Print results
print("Status:", LpStatus[modelb.status])
print("Optimal Solution:")
for p in products:
    for m in months:
        print(f"{p} - {m}: Produce = {P[p, m].varValue}, Inventory = {I[p, m].varValue}")

# Calculate total cost for each product
product_costs = {p: 0 for p in products}

# Add production and holding costs for each product
for p in products:
    production_cost_sum = sum(
        production_cost[p] * P[p, m].varValue for m in months
    )
    holding_cost_sum = sum(
        holding_cost[p] * I[p, m].varValue for m in months
    )
    # Adjust for specific product and month cost if needed
    if p == "B":
        production_cost_sum += 5 * P["B", "3"].varValue  # Adjusted cost for P_B_3

    # Store total cost for the product
    product_costs[p] = production_cost_sum + holding_cost_sum

# Print total costs for each product
print("\nTotal Cost Breakdown by Product after adjusting P_B_3:")
for p in products:
    print(f"Product {p}: £{product_costs[p]:.2f}")

# Optional: Print the overall total cost (should match model.objective.value())
overall_cost = sum(product_costs[p] for p in products)
print(f"\nOverall Total Cost: £{overall_cost:.2f}")

Status: Optimal
Optimal Solution:
A - 1: Produce = 100.0, Inventory = 0.0
A - 2: Produce = 110.0, Inventory = 0.0
A - 3: Produce = 120.0, Inventory = 0.0
A - 4: Produce = 130.0, Inventory = 0.0
B - 1: Produce = 80.0, Inventory = 0.0
B - 2: Produce = 190.0, Inventory = 100.0
B - 3: Produce = 0.0, Inventory = 0.0
B - 4: Produce = 110.0, Inventory = 0.0
C - 1: Produce = 179.0, Inventory = 59.0
C - 2: Produce = 71.0, Inventory = 0.0
C - 3: Produce = 239.0, Inventory = 99.0
C - 4: Produce = 51.0, Inventory = 0.0

Total Cost Breakdown by Product after adjusting P_B_3:
Product A: £9200.00
Product B: £11600.00
Product C: £5479.00

Overall Total Cost: £26279.00


## Question 1.c

In [ ]:
# Re-solve the model
model = LpProblem("Question1_a_updated", LpMinimize)

# Re-define the Objective Function
model += lpSum(production_cost[p] * P[p, m] + holding_cost[p] * I[p, m] for p in products for m in months)

# Update the demand for product C in month 4
demand["4"]["C"] += 20  # Increase demand by 20 units

# Re-define Constraints
for p in products:
    for i, m in enumerate(months):
        prev_inventory = I[p, months[i - 1]] if i > 0 else 0
        model += P[p, m] + prev_inventory == demand[m][p] + I[p, m]

for m in months:
    model += lpSum(production_hours[p] * P[p, m] for p in products) <= available_hours[m]

for p in products:
    for m in months:
        if m != "4":
            model += P[p, m] >= 0
            model += I[p, m] >= 0

# Solve the updated model
model.solve()

# Print updated results
print("Status:", LpStatus[model.status])
print("Updated Optimal Solution:")
for p in products:
    for m in months:
        print(f"{p} - {m}: Produce = {P[p, m].varValue}, Inventory = {I[p, m].varValue}")

# Add production and holding costs for each product
for p in products:
    production_cost_sum = sum(
        production_cost[p] * P[p, m].varValue for m in months
    )
    holding_cost_sum = sum(
        holding_cost[p] * I[p, m].varValue for m in months
    )
    # Adjust for specific product and month cost if needed

    # Store total cost for the product
    product_costs[p] = production_cost_sum + holding_cost_sum

# Print total costs for each product
print("\nTotal Cost Breakdown by Product:")
for p in products:
    print(f"Product {p}: £{product_costs[p]:.2f}")

# Optional: Print the overall total cost (should match model.objective.value())
overall_cost = sum(product_costs[p] for p in products)
print(f"\nOverall Total Cost: £{overall_cost:.2f}")

Status: Optimal
Updated Optimal Solution:
A - 1: Produce = 100.0, Inventory = 0.0
A - 2: Produce = 110.0, Inventory = 0.0
A - 3: Produce = 120.0, Inventory = 0.0
A - 4: Produce = 130.0, Inventory = 0.0
B - 1: Produce = 80.0, Inventory = 0.0
B - 2: Produce = 90.0, Inventory = 0.0
B - 3: Produce = 100.0, Inventory = 0.0
B - 4: Produce = 110.0, Inventory = 0.0
C - 1: Produce = 182.0, Inventory = 62.0
C - 2: Produce = 201.0, Inventory = 133.0
C - 3: Produce = 126.0, Inventory = 119.0
C - 4: Produce = 51.0, Inventory = 0.0

Total Cost Breakdown by Product:
Product A: £9200.00
Product B: £11400.00
Product C: £5757.00

Overall Total Cost: £26357.00
Total Cost: £26357.0


In [ ]:
model

Question1_a_updated:
MINIMIZE
1*I_A_1 + 1*I_A_2 + 1*I_A_3 + 1*I_A_4 + 2*I_B_1 + 2*I_B_2 + 2*I_B_3 + 2*I_B_4 + 0.5*I_C_1 + 0.5*I_C_2 + 0.5*I_C_3 + 0.5*I_C_4 + 20*P_A_1 + 20*P_A_2 + 20*P_A_3 + 20*P_A_4 + 30*P_B_1 + 30*P_B_2 + 30*P_B_3 + 30*P_B_4 + 10*P_C_1 + 10*P_C_2 + 10*P_C_3 + 10*P_C_4 + 0.0
SUBJECT TO
_C1: - I_A_1 + P_A_1 = 100

_C2: I_A_1 - I_A_2 + P_A_2 = 110

_C3: I_A_2 - I_A_3 + P_A_3 = 120

_C4: I_A_3 - I_A_4 + P_A_4 = 130

_C5: - I_B_1 + P_B_1 = 80

_C6: I_B_1 - I_B_2 + P_B_2 = 90

_C7: I_B_2 - I_B_3 + P_B_3 = 100

_C8: I_B_3 - I_B_4 + P_B_4 = 110

_C9: - I_C_1 + P_C_1 = 120

_C10: I_C_1 - I_C_2 + P_C_2 = 130

_C11: I_C_2 - I_C_3 + P_C_3 = 140

_C12: I_C_3 - I_C_4 + P_C_4 = 170

_C13: 1.2 P_A_1 + 1.3 P_B_1 + P_C_1 <= 500

_C14: 1.2 P_A_2 + 1.3 P_B_2 + P_C_2 <= 450

_C15: 1.2 P_A_3 + 1.3 P_B_3 + P_C_3 <= 400

_C16: 1.2 P_A_4 + 1.3 P_B_4 + P_C_4 <= 350

_C17: P_A_1 >= 0

_C18: I_A_1 >= 0

_C19: P_A_2 >= 0

_C20: I_A_2 >= 0

_C21: P_A_3 >= 0

_C22: I_A_3 >= 0

_C23: P_B_1 >= 0

_C

## Question 1.d

In [ ]:
# Define the problem
model = LpProblem("Question1_d", LpMinimize)

# Parameters
products = ["A", "B", "C"]
months = ["1", "2", "3", "4"]
production_cost = {"A": 20, "B": 30, "C": 10}
holding_cost = {"A": 1, "B": 2, "C": 0.5}
production_hours = {"A": 1.2, "B": 1.3, "C": 1}
available_hours = {"1": 500, "2": 450, "3": 400, "4": 350}
demand = {
    "1": {"A": 100, "B": 80, "C": 120},
    "2": {"A": 110, "B": 90, "C": 130},
    "3": {"A": 120, "B": 100, "C": 140},
    "4": {"A": 130, "B": 110, "C": 150},
}

# Decision Variables
P = {(p, m): LpVariable(f"P_{p}_{m}", lowBound=0, cat="Integer") for p in products for m in months}
I = {(p, m): LpVariable(f"I_{p}_{m}", lowBound=0, cat="Continuous") for p in products for m in months}

# Objective Function: Minimize total production and holding costs
model += lpSum(production_cost[p] * P[p, m] + holding_cost[p] * I[p, m] for p in products for m in months)

# Constraints
# Production and Inventory Balance (strict equality)
for p in products:
    for i, m in enumerate(months):
        prev_inventory = I[p, months[i - 1]] if i > 0 else 0
        model += P[p, m] + prev_inventory == demand[m][p] + I[p, m]

# Production Hours Limit
for m in months:
    model += lpSum(production_hours[p] * P[p, m] for p in products) <= available_hours[m]

# Inventory Holding cost not exceeding £500
model += lpSum(holding_cost[p] * I[p, m] for p in products for m in months) <= 500

# Solve the problem
model.solve()

# Print results
print("Status:", LpStatus[model.status])
print("Optimal Solution:")
for p in products:
    for m in months:
        print(f"{p} - {m}: Produce = {P[p, m].varValue}, Inventory = {I[p, m].varValue}")

# Calculate total costs for each product
product_costs = {p: 0 for p in products}

for p in products:
    production_cost_sum = sum(
        production_cost[p] * P[p, m].varValue for m in months
    )
    holding_cost_sum = sum(
        holding_cost[p] * I[p, m].varValue for m in months
    )
    product_costs[p] = production_cost_sum + holding_cost_sum

# Print total costs for each product
print("\nTotal Cost Breakdown by Product:")
for p in products:
    print(f"Product {p}: £{product_costs[p]:.2f}")

# Calculate and print total holding cost
total_holding_cost = sum(
    holding_cost[p] * I[p, m].varValue for p in products for m in months
)
print(f"\nTotal Holding Cost: £{total_holding_cost:.2f}")

# Optional: Print the overall total cost (should match model.objective.value())
overall_cost = sum(product_costs[p] for p in products)
print(f"\nOverall Total Cost: £{overall_cost:.2f}")


Status: Optimal
Optimal Solution:
A - 1: Produce = 100.0, Inventory = 0.0
A - 2: Produce = 110.0, Inventory = 0.0
A - 3: Produce = 120.0, Inventory = 0.0
A - 4: Produce = 130.0, Inventory = 0.0
B - 1: Produce = 80.0, Inventory = 0.0
B - 2: Produce = 90.0, Inventory = 0.0
B - 3: Produce = 100.0, Inventory = 0.0
B - 4: Produce = 110.0, Inventory = 0.0
C - 1: Produce = 162.0, Inventory = 42.0
C - 2: Produce = 201.0, Inventory = 113.0
C - 3: Produce = 126.0, Inventory = 99.0
C - 4: Produce = 51.0, Inventory = 0.0

Total Cost Breakdown by Product:
Product A: £9200.00
Product B: £11400.00
Product C: £5527.00

Total Holding Cost: £127.00

Overall Total Cost: £26127.00
